In [1]:
# Import libraries
import datetime as dt
import numpy as np
import os
import pandas as pd

from IPython.display import display

In [2]:
os.chdir('RC_2021_results')

In [3]:
# Dataset metadata
datasets = ['labour', 'traffic', 'tourism', 'tourismlarge', 'wiki']
levels_dict = {'labour': 4,
               'tourism': 4,
               'tourismlarge': {'geo': 4,
                                'trav': 5},
               'traffic': 4,
               'wiki': 5}


# Hyperparameter configuration
epoch_set = [10, 25, 50]
context_len_dict = {'tourism': [2, 3, 4],
                    'tourismlarge': [2, 3, 4],
                    'labour': [2, 3, 4],
                    'traffic': [15, 25, 40, 60],
                    'wiki': [15, 25, 40, 60]}

optimal_hps = {'DeepVAR': {'traffic': {'epochs': 25,
                                       'context_len_factor': 40},
                           'wiki': {'epochs': 25,
                                    'context_len_factor': 15},
                           'labour': {'epochs': 50,
                                      'context_len_factor': 3},
                           'tourism': {'epochs': 10,
                                       'context_len_factor': 3},
                           'tourismlarge': {'epochs': 10,
                                            'context_len_factor': 3}},
               'DeepVARPlus': {'traffic': {'epochs': 25,
                                           'context_len_factor': 40},
                               'wiki': {'epochs': 25,
                                        'context_len_factor': 15},
                               'labour': {'epochs': 50,
                                          'context_len_factor': 3},
                               'tourism': {'epochs': 10,
                                           'context_len_factor': 2},
                               'tourismlarge': {'epochs': 25,
                                                'context_len_factor': 3}},
              'HierE2E': {'traffic': {'epochs': 50,
                                      'context_len_factor': 40},
                          'wiki': {'epochs': 50,
                                   'context_len_factor': 15},
                          'labour': {'epochs': 50,
                                     'context_len_factor': 3},
                          'tourism': {'epochs': 10,
                                      'context_len_factor': 3},
                          'tourismlarge': {'epochs': 50,  
                                           'context_len_factor': 3}}}


# Additional parameters
num_runs = 5
mean_col = f'mean_of_{num_runs}_runs'
methods = ['ARIMA_NaiveBU', 'ETS_NaiveBU', 'ARIMA_MINT_shr', 'ARIMA_MINT_ols', 'ETS_MINT_shr', 'ETS_MINT_ols',
           'ARIMA_ERM', 'ETS_ERM', 'PERMBU_MINT', 'HierE2E', 'DeepVAR', 'DeepVARPlus']
classical_methods = [method for method in methods if method not in ['DeepVAR', 'DeepVARPlus', 'HierE2E']]
neural_methods = ['HierE2E', 'DeepVAR', 'DeepVARPlus']

### Helper functions

In [17]:
# Function which creates a list of the names of the losses we're interested in for a particular dataset
def get_cols_of_interest(dataset):
    if dataset != 'tourismlarge':
        cols_of_interest = ['mean_wQuantileLoss'] + \
                           [f'level_{i}_mean_wQuantileLoss' for i in range (1, levels_dict[dataset] + 1)]
    else:
        cols_of_interest = ['mean_wQuantileLoss', 'level_1_mean_wQuantileLoss'] + \
                           [f'level_{i}_geo_mean_wQuantileLoss' for i in range (2, levels_dict[dataset]['geo'] + 1)] + \
                           [f'level_{i}_trav_mean_wQuantileLoss' for i in range (2, levels_dict[dataset]['trav'] + 1)]
        
    return cols_of_interest

def select_pickles(folder_name):
    files = [file for file in os.listdir(folder_name) if '.txt' not in file]
    return files

# Function which evaluates the results for a particular model on a particular dataset
def get_model_results(folder_name, dataset):
    # we get the losses of interest for a particular dataset
    cols_of_interest = get_cols_of_interest(dataset)
    
    if folder_name == os.path.join('PERMBU_MINT', 'tourismlarge'):
        best_results = pd.DataFrame(len(cols_of_interest) * ['NA'], index=cols_of_interest).T
        best_results.index = ['PERMBU_MINT']
        return best_results
    
    # we store the results from the separate runs into a dataframe
    best_results = pd.DataFrame()
    files = select_pickles(folder_name)
    for (i, file) in enumerate(files):
        losses = np.empty((len(cols_of_interest),))
        results = pd.read_pickle(os.path.join(folder_name, file))
        results = pd.DataFrame(results) 

        for (j, col) in enumerate(cols_of_interest):
            if col == 'mean_wQuantileLoss':
                losses[j] = results.loc[0, col]
            else:
                losses[j] = results.loc[1, col]

        selected_results = pd.DataFrame(losses, index=cols_of_interest, columns=[f'run_{i}'])
        best_results = pd.concat([best_results, selected_results], axis=1)
    
    # we add the mean (overall and hierarchical ones) of all runs
    best_results[mean_col] = best_results.mean(axis=1)
    
    return best_results

# Function to calculate the average loss across multiple runs for a particular hyperparameter configuration
def get_mean_loss(folder_name, num_runs, calc_std_error):
    losses = np.empty((num_runs,))
    files = select_pickles(folder_name)
    for (i, file) in enumerate(files):
        results = pd.read_pickle(os.path.join(folder_name, file))
        results = pd.DataFrame(results)

        losses[i] = results.loc[0, 'mean_wQuantileLoss']
            
    mean_loss = np.mean(losses)
    if calc_std_error == True:
        loss_std_error = np.std(losses)
        return mean_loss, loss_std_error
    else:
        return mean_loss

# Function which evaluates the performance of a method over the entire hyperparameter space on a given dataset
def get_method_results_on_dataset(model, dataset):
    # we get the losses of interest for the dataset
    cols_of_interest = get_cols_of_interest(dataset)
    
    # we search (based on the results) for the best hyperparameter configuration
    # for the method on a given dataset
    folder_name = model + '_' + dataset 
    min_loss = 1000  # just a random loss to initialize the variable
    optimal_epochs = 0
    optimal_context_len = 0
    for epochs in epoch_set:
        for context_len in context_len_dict[dataset]:
            subfolder_name = os.path.join(folder_name, f'epochs_{epochs}_context_len_factor_{context_len}')
            assert len(os.listdir(subfolder_name)) == num_runs, \
                   f"You didn't run the model epochs_{epochs}_context_len_factor_{context_len} for {dataset} {num_runs} times"
            
            # we check if the mean loss (across num_runs runs) for a given configuration 
            # is lower than the one of the most optimal configuration so far
            mean_loss = get_mean_loss(subfolder_name, num_runs, calc_std_error=False)
            if mean_loss < min_loss:
                min_loss = mean_loss
                optimal_epochs = epochs
                optimal_context_len = context_len
    
    # we store the results of the most optimal hyperparameter configuration into a dataframe
    subfolder_name = os.path.join(folder_name, f'epochs_{optimal_epochs}_context_len_factor_{optimal_context_len}')
    best_results = get_model_results(subfolder_name, dataset)
    
    return optimal_epochs, optimal_context_len, best_results

# Function which evaluates the performance of a method over the entire hyperparameter space on all datasets
def get_method_results_on_datasets(model):
    for dataset in ['wiki']: #datasets:
        optimal_epochs, optimal_context_len, best_results = get_method_results_on_dataset(model, dataset)
        print(f'Method {model} on dataset {dataset}:\n\t')
        print(f'Most optimal number of epochs: {optimal_epochs}\n\t')
        print(f'Most optimal context length factor: {optimal_context_len}\n\t')
        print('The performance of this configuration:\n')
        display(best_results)

def correct_folder_names(folder_name):
    if 'tourism' in folder_name and 'large' not in folder_name:
        folder_name = folder_name.replace('tourism', 'tourismsmall')
    if 'wiki' in folder_name:
        folder_name = folder_name.replace('wiki', 'wiki2')
        
    return folder_name

def get_paper_results(model, datasets, display_results=False):
    for dataset in datasets:
        folder_name = os.path.join(model, dataset)
        
        folder_name = correct_folder_names(folder_name)
        
        dataset_results = get_model_results(folder_name, dataset)
        
        if display_results:
            print(f'Performance of model {model} on dataset {dataset}:\n')
            display(dataset_results)
            print('\n\n')
        
    if len(datasets) == 1:
        return dataset_results

def create_paper_table_1(verbose=True, to_latex=False):
    results_df = pd.DataFrame()
    for method in methods:
        method_df = pd.DataFrame(index=[method])
        for dataset in datasets:
            if verbose:
                print(f'Adding results for method {method} on dataset {dataset}..\n')
            
            folder_name = os.path.join(method, dataset)
            folder_name = correct_folder_names(folder_name)

            if folder_name == os.path.join('PERMBU_MINT', 'tourismlarge'):
                method_df[dataset] = 'NA'
            else:
                mean_loss, loss_std_error = get_mean_loss(folder_name, num_runs, calc_std_error=True)
                method_df[dataset] = str(np.round(mean_loss, 4)) + ' +/- ' + str(np.round(loss_std_error, 4))

        results_df = pd.concat([results_df, method_df])
        
    if to_latex:
        results_df.to_latex('paper_table_1.tex')
    
    return results_df

def examine_hp_search_results(method, dataset, num_runs):
    optimal_epochs = optimal_hps[method][dataset]['epochs']
    optimal_context_len_factor = optimal_hps[method][dataset]['context_len_factor']
    
    folder_name = os.path.join(f'{method}_{dataset}', 
                               f'epochs_{optimal_epochs}_context_len_factor_{optimal_context_len_factor}')
    mean_loss, loss_std_error = get_mean_loss(folder_name, num_runs, calc_std_error=True)
    print(f'Method {method} on dataset {dataset}:\n')
    print(f'epochs_{optimal_epochs}_context_len_factor_{optimal_context_len_factor}:\n')
    print(f'mean_loss: {mean_loss}\n')
    print(f'loss_std_error: {loss_std_error}\n')
    print(f'Results across {num_runs} runs for this hyperparameter configuration:\n')
    display(get_model_results(folder_name, dataset))
    print('\n\n')

    my_optimal_epochs, my_optimal_context_len_factor, best_results = get_method_results_on_dataset(method, dataset)
    print(f'Results across {num_runs} runs for the configuration we found to be the best:\n')
    display(best_results)
    print(f'optimal_epochs for this configuration: {my_optimal_epochs}\n')
    print(f'optimal_context_len_factor for this configuration: {my_optimal_context_len_factor}\n\n')

def get_methods_results(dataset):
    dataset_df = pd.DataFrame()
    for method in methods:
        method_df = get_paper_results(method, [dataset])
        if (method == 'PERMBU_MINT') and (dataset == 'tourismlarge'):
            dataset_df = pd.concat([dataset_df, method_df])
        else:
            std_col = method_df.loc[:, method_df.columns != mean_col].std(axis=1).round(4)
            method_df[mean_col] = method_df[mean_col].round(4)
            method_df[mean_col] = method_df[mean_col].astype('str') + ' +/- ' + std_col.astype(str)
            dataset_df = pd.concat([dataset_df, method_df[mean_col].to_frame().T])
    return dataset_df

def get_supplementary_table(dataset, to_latex=False):
    dataset_df = get_methods_results(dataset)
    dataset_df.index = methods
    dataset_df.columns = [(col.split('wQuantileLoss')[0] + 'CRPS') for col in dataset_df.columns]
    
    if to_latex:
        dataset_df.to_latex(f'{dataset}_supplementary_table.tex')
    return dataset_df

def split_result(x):
    if x == 'NA':
        return 'NA'
    else:
        return float(x.split(' +/- ')[0])
    
def get_supplementary_subtable(dataset, to_latex=False):
    complete_dataset_df = get_supplementary_table(dataset)
    dataset_df = complete_dataset_df.applymap(split_result)
    
    count_dict = dict.fromkeys(classical_methods, 0)
    dataset_df_tmp = dataset_df.iloc[:, 1:].loc[dataset_df.index.isin(classical_methods), :]
    if dataset == 'tourismlarge':
        dataset_df_tmp = dataset_df_tmp.drop('PERMBU_MINT')
    dataset_df_tmp = dataset_df_tmp.astype('float64')
    for col in dataset_df_tmp.columns:
        best_method = dataset_df_tmp.loc[:, col].idxmin()
        count_dict[best_method] += 1
    
    max_num_levels = max(count_dict.values())
    best_methods = [method for method, num in count_dict.items() if num == max_num_levels]
    if len(best_methods) > 1:
        best_methods = [dataset_df.loc[best_methods, 'mean_CRPS'].idxmin()]
    complete_dataset_df = complete_dataset_df.loc[best_methods + neural_methods, :]
    
    complete_dataset_df = complete_dataset_df.T
    if to_latex:
        complete_dataset_df.to_latex(f'{dataset}_supplementary_subtable.tex')
    return complete_dataset_df

def convert_date(file):
    date = file.split('run_')[1].replace('.pkl', '').split('_')
    date = '/'.join(date[: 3]) + ' ' + ':'.join(date[3:])
    date = dt.datetime.strptime(date, '%Y/%m/%d %H:%M:%S')
    return date

def calculate_runtimes(folder_name):
    runtimes = np.zeros((num_runs - 1,))
    files = [file for file in os.listdir(folder_name) if '.txt' not in file]
    files = sorted(files)
    for (i, file) in enumerate(files):
        if i == 0:
            old_date = convert_date(file)
        else:
            new_date = convert_date(file)
            runtimes[i-1] = (new_date - old_date).seconds
            old_date = new_date
    return runtimes

def calculate_average_runtimes(to_latex=False, calc_std_error=False, minutes=False):
    runtime_dict = dict.fromkeys(methods, dict.fromkeys(datasets))
    runtime_df = pd.DataFrame(runtime_dict)

    for method in methods:
        for dataset in datasets:
            folder_name = os.path.join(method, dataset)
            folder_name = correct_folder_names(folder_name)
            if ('PERMBU_MINT' in folder_name) and ('tourismlarge' in folder_name):
                continue
            runtimes = calculate_runtimes(folder_name)
            if calc_std_error and minutes:
                runtime_df.loc[dataset, method] = str((runtimes.mean() / 60).round(2)) + ' +/- ' + str((runtimes.std() / 60).round(2))
            else:
                runtime_df.loc[dataset, method] = runtimes.mean()
                
    if to_latex:
        runtime_df.T.to_latex('runtime_table.tex')
    return runtime_df.T

def calculate_total_time():
    runtime_df = calculate_average_runtimes()
    runtime_df = runtime_df.sum(axis=1) * num_runs  
    for method in neural_methods:
        for dataset in datasets:
            folder_name = method + '_' + dataset
            subfolders = [subfolder for subfolder in os.listdir(folder_name) if ('.txt' not in subfolder) and ('.ipynb_checkpoints' not in subfolder)]
            subfolders = [os.path.join(folder_name, subfolder) for subfolder in subfolders]
            for subfolder in subfolders:
                runtime_df.loc[method] += calculate_runtimes(subfolder)[: 3].mean() * num_runs
                
    runtime_df = runtime_df.to_frame()
    runtime_df.columns = ['total_time [h]']
    runtime_df = (runtime_df / 3600).round(2)
    return runtime_df

## Comparison of performance of authors' optimal configurations against mine

**Best configurations**:

- *DeepVAR*: 
    - traffic: (found: 50, 60) (paper: 25, 40)
    - wiki: (found: 10, 15) (paper: 25, 15)
    - labour: (found: 25, 2) (paper: 50, 3)
    - tourism: (found: 10, 4) (paper: 10, 3)
    - tourismlarge: (found: 10, 2) (paper: 10, 3)
    
    
- *DeepVARPlus*:
    - traffic: (found: 25, 60) (paper: 25, 40)
    - wiki: (found: 10, 15) (paper: 25, 15)
    - labour: (found: 50, 3) (paper: 50, 3)
    - tourism: (found: 10, 4) (paper: 10, 2)
    - tourismlarge: (found: 50, 4) (paper: 25, 3)
    
    
- *HierE2E*:
    - traffic: (found: 50, 60) (paper: 50, 40)
    - wiki: (found: 50, 15) (paper: 50, 15)
    - labour: (found: 50, 2) (paper: 50, 3)
    - tourism: (found: 10, 3) (paper: 10, 3)
    - tourismlarge: (found: 50, 4) (paper: 50, 3)

In [8]:
for method in neural_methods:
    for dataset in datasets:
        examine_hp_search_results(method, dataset, num_runs)

Method HierE2E on dataset labour:

epochs_50_context_len_factor_3:

mean_loss: 0.014300808423089926

loss_std_error: 0.0015778704851996987

Results across 5 runs for this hyperparameter configuration:



,run_0,run_1,run_2,run_3,run_4,mean_of_5_runs
mean_wQuantileLoss,0.016347,0.012833,0.012536,0.015963,0.013825,0.014301
level_1_mean_wQuantileLoss,0.013146,0.007103,0.004938,0.012752,0.009868,0.009562
level_2_mean_wQuantileLoss,0.016205,0.012384,0.011673,0.014966,0.013120,0.013670
level_3_mean_wQuantileLoss,0.016418,0.013549,0.013822,0.016075,0.014256,0.014824
level_4_mean_wQuantileLoss,0.019619,0.018294,0.019711,0.020061,0.018056,0.019148





Results across 5 runs for the configuration we found to be the best:



,run_0,run_1,run_2,run_3,run_4,mean_of_5_runs
mean_wQuantileLoss,0.013594,0.010111,0.010189,0.011972,0.012797,0.011733
level_1_mean_wQuantileLoss,0.010769,0.005716,0.005274,0.007772,0.009029,0.007712
level_2_mean_wQuantileLoss,0.012021,0.007807,0.009091,0.010097,0.011514,0.010106
level_3_mean_wQuantileLoss,0.012987,0.010313,0.010322,0.011850,0.012422,0.011579
level_4_mean_wQuantileLoss,0.018599,0.016608,0.016069,0.018170,0.018224,0.017534


optimal_epochs for this configuration: 50

optimal_context_len_factor for this configuration: 2


Method HierE2E on dataset traffic:

epochs_50_context_len_factor_40:

mean_loss: 0.04742799064618816

loss_std_error: 0.002065178424655306

Results across 5 runs for this hyperparameter configuration:



...

optimal_epochs for this configuration: 50

optimal_context_len_factor for this configuration: 4


Method DeepVARPlus on dataset wiki:

epochs_25_context_len_factor_15:

mean_loss: 0.2377701548073529

loss_std_error: 0.013063235740950275

Results across 5 runs for this hyperparameter configuration:



,run_0,run_1,run_2,run_3,run_4,mean_of_5_runs
mean_wQuantileLoss,0.224226,0.251645,0.251481,0.220984,0.240516,0.237770
level_1_mean_wQuantileLoss,0.118369,0.147537,0.130533,0.118306,0.145122,0.131974
level_2_mean_wQuantileLoss,0.161302,0.212805,0.185371,0.166594,0.192328,0.183680
level_3_mean_wQuantileLoss,0.216701,0.235410,0.254482,0.199255,0.229402,0.227050
level_4_mean_wQuantileLoss,0.247812,0.271763,0.286035,0.231376,0.257135,0.258824
level_5_mean_wQuantileLoss,0.376945,0.390707,0.400981,0.389389,0.378593,0.387323





Results across 5 runs for the configuration we found to be the best:



,run_0,run_1,run_2,run_3,run_4,mean_of_5_runs
mean_wQuantileLoss,0.165791,0.179253,0.159620,0.140375,0.168302,0.162668
level_1_mean_wQuantileLoss,0.073595,0.070772,0.069625,0.055193,0.062647,0.066367
level_2_mean_wQuantileLoss,0.123042,0.141984,0.111266,0.092725,0.116686,0.117140
level_3_mean_wQuantileLoss,0.158032,0.173699,0.154496,0.125709,0.157759,0.153939
level_4_mean_wQuantileLoss,0.179037,0.196572,0.174403,0.146643,0.183262,0.175983
level_5_mean_wQuantileLoss,0.295251,0.313240,0.288310,0.281605,0.321156,0.299912


optimal_epochs for this configuration: 10

optimal_context_len_factor for this configuration: 15




## Table 4 from Reproducibility Report

In [9]:
display(create_paper_table_1(to_latex=False))

Adding results for method ARIMA_NaiveBU on dataset labour..

Adding results for method ARIMA_NaiveBU on dataset traffic..

Adding results for method ARIMA_NaiveBU on dataset tourism..

Adding results for method ARIMA_NaiveBU on dataset tourismlarge..

Adding results for method ARIMA_NaiveBU on dataset wiki..

Adding results for method ETS_NaiveBU on dataset labour..

Adding results for method ETS_NaiveBU on dataset traffic..

Adding results for method ETS_NaiveBU on dataset tourism..

Adding results for method ETS_NaiveBU on dataset tourismlarge..

Adding results for method ETS_NaiveBU on dataset wiki..

Adding results for method ARIMA_MINT_shr on dataset labour..

Adding results for method ARIMA_MINT_shr on dataset traffic..

Adding results for method ARIMA_MINT_shr on dataset tourism..

Adding results for method ARIMA_MINT_shr on dataset tourismlarge..

Adding results for method ARIMA_MINT_shr on dataset wiki..

Adding results for method ARIMA_MINT_ols on dataset labour..

Adding res

,labour,traffic,tourism,tourismlarge,wiki
ARIMA_NaiveBU,0.0453 +/- 0.0,0.0753 +/- 0.0,0.1138 +/- 0.0,0.1752 +/- 0.0,0.3776 +/- 0.0
ETS_NaiveBU,0.0432 +/- 0.0,0.0665 +/- 0.0,0.1008 +/- 0.0,0.169 +/- 0.0,0.4673 +/- 0.0
ARIMA_MINT_shr,0.0467 +/- 0.0,0.0775 +/- 0.0,0.1171 +/- 0.0,0.1615 +/- 0.0,0.2466 +/- 0.0
ARIMA_MINT_ols,0.0463 +/- 0.0,0.1123 +/- 0.0,0.1195 +/- 0.0,0.1731 +/- 0.0,0.2782 +/- 0.0
ETS_MINT_shr,0.0455 +/- 0.0,0.0963 +/- 0.0,0.1013 +/- 0.0,0.1627 +/- 0.0,0.3622 +/- 0.0
ETS_MINT_ols,0.0459 +/- 0.0,0.111 +/- 0.0,0.1002 +/- 0.0,0.1668 +/- 0.0,0.2702 +/- 0.0
ARIMA_ERM,0.0399 +/- 0.0,0.0466 +/- 0.0,0.5885 +/- 0.0,0.5668 +/- 0.0,0.2195 +/- 0.0
ETS_ERM,0.0456 +/- 0.0,0.1027 +/- 0.0,2.3742 +/- 0.0,0.508 +/- 0.0,0.2217 +/- 0.0
PERMBU_MINT,0.0393 +/- 0.0002,0.0679 +/- 0.0047,0.0763 +/- 0.0003,NA,0.279 +/- 0.02
HierE2E,0.0335 +/- 0.0064,0.0359 +/- 0.0114,0.0916 +/- 0.0082,0.1688 +/- 0.0036,0.1629 +/- 0.0056


## Table 5 from Reproducibility Report (separated in subtables)

In [10]:
for dataset in ['tourism', 'tourismlarge', 'labour', 'traffic', 'wiki']:
    print(f'\n\nThis is a table displaying the CRPS score across levels\non the dataset {dataset} for the optimal configuration of\nHierE2E, DeepVAR, DeepVAR+ and the best classical machine learning approach: \n')
    display(get_supplementary_subtable(dataset, to_latex=False))



This is a table displaying the CRPS score across levels
on the dataset tourism for the optimal configuration of
HierE2E, DeepVAR, DeepVAR+ and the best classical machine learning approach: 



,PERMBU_MINT,HierE2E,DeepVAR,DeepVARPlus
mean_CRPS,0.0763 +/- 0.0003,0.0916 +/- 0.0091,0.0953 +/- 0.0062,0.0956 +/- 0.018
level_1_mean_CRPS,0.0464 +/- 0.0017,0.051 +/- 0.0099,0.0531 +/- 0.012,0.0509 +/- 0.019
level_2_mean_CRPS,0.0592 +/- 0.0008,0.0765 +/- 0.0113,0.0827 +/- 0.0091,0.0776 +/- 0.0216
level_3_mean_CRPS,0.0899 +/- 0.0011,0.1104 +/- 0.008,0.112 +/- 0.0086,0.1148 +/- 0.018
level_4_mean_CRPS,0.1097 +/- 0.0009,0.1286 +/- 0.0079,0.1333 +/- 0.0062,0.139 +/- 0.0152




This is a table displaying the CRPS score across levels
on the dataset tourismlarge for the optimal configuration of
HierE2E, DeepVAR, DeepVAR+ and the best classical machine learning approach: 



,ARIMA_MINT_shr,HierE2E,DeepVAR,DeepVARPlus
mean_CRPS,0.1615 +/- 0.0,0.1688 +/- 0.004,0.1394 +/- 0.0021,0.1979 +/- 0.0294
level_1_mean_CRPS,0.0443 +/- 0.0,0.0959 +/- 0.0105,0.0634 +/- 0.005,0.1234 +/- 0.043
level_2_geo_mean_CRPS,0.0826 +/- 0.0,0.1161 +/- 0.0063,0.0814 +/- 0.0029,0.1417 +/- 0.0351
level_3_geo_mean_CRPS,0.1439 +/- 0.0,0.1503 +/- 0.0053,0.1216 +/- 0.003,0.1775 +/- 0.0304
level_4_geo_mean_CRPS,0.2042 +/- 0.0,0.1901 +/- 0.0045,0.1629 +/- 0.0017,0.218 +/- 0.0263
level_2_trav_mean_CRPS,0.0834 +/- 0.0,0.1209 +/- 0.0039,0.0891 +/- 0.0087,0.1464 +/- 0.0331
level_3_trav_mean_CRPS,0.1485 +/- 0.0,0.1619 +/- 0.0044,0.1302 +/- 0.004,0.1895 +/- 0.0259
level_4_trav_mean_CRPS,0.244 +/- 0.0,0.2242 +/- 0.0044,0.1979 +/- 0.0012,0.2556 +/- 0.0234
level_5_trav_mean_CRPS,0.3413 +/- 0.0,0.2913 +/- 0.0053,0.2684 +/- 0.0026,0.3314 +/- 0.0245




This is a table displaying the CRPS score across levels
on the dataset labour for the optimal configuration of
HierE2E, DeepVAR, DeepVAR+ and the best classical machine learning approach: 



,PERMBU_MINT,HierE2E,DeepVAR,DeepVARPlus
mean_CRPS,0.0393 +/- 0.0003,0.0335 +/- 0.0072,0.0367 +/- 0.0055,0.0457 +/- 0.013
level_1_mean_CRPS,0.0406 +/- 0.0004,0.0302 +/- 0.0093,0.0342 +/- 0.005,0.0445 +/- 0.016
level_2_mean_CRPS,0.0388 +/- 0.0003,0.0342 +/- 0.0071,0.0362 +/- 0.0059,0.0461 +/- 0.013
level_3_mean_CRPS,0.0382 +/- 0.0002,0.0335 +/- 0.0066,0.0362 +/- 0.0056,0.0456 +/- 0.0125
level_4_mean_CRPS,0.0396 +/- 0.0003,0.0361 +/- 0.0058,0.0403 +/- 0.0067,0.0466 +/- 0.0106




This is a table displaying the CRPS score across levels
on the dataset traffic for the optimal configuration of
HierE2E, DeepVAR, DeepVAR+ and the best classical machine learning approach: 



,ARIMA_ERM,HierE2E,DeepVAR,DeepVARPlus
mean_CRPS,0.0466 +/- 0.0,0.0359 +/- 0.0127,0.0334 +/- 0.0036,0.0366 +/- 0.0088
level_1_mean_CRPS,0.0089 +/- 0.0,0.0166 +/- 0.017,0.0131 +/- 0.0058,0.013 +/- 0.0081
level_2_mean_CRPS,0.0113 +/- 0.0,0.0178 +/- 0.0159,0.0174 +/- 0.0121,0.0158 +/- 0.008
level_3_mean_CRPS,0.0254 +/- 0.0,0.0186 +/- 0.0154,0.0198 +/- 0.0086,0.0209 +/- 0.0124
level_4_mean_CRPS,0.1408 +/- 0.0,0.0905 +/- 0.0061,0.0835 +/- 0.0027,0.0969 +/- 0.0096




This is a table displaying the CRPS score across levels
on the dataset wiki for the optimal configuration of
HierE2E, DeepVAR, DeepVAR+ and the best classical machine learning approach: 



,ETS_ERM,HierE2E,DeepVAR,DeepVARPlus
mean_CRPS,0.2217 +/- 0.0,0.1629 +/- 0.0063,0.2081 +/- 0.0067,0.2053 +/- 0.0146
level_1_mean_CRPS,0.1558 +/- 0.0,0.0668 +/- 0.0056,0.0751 +/- 0.0153,0.0523 +/- 0.0158
level_2_mean_CRPS,0.1614 +/- 0.0,0.1184 +/- 0.0062,0.1199 +/- 0.0143,0.1053 +/- 0.009
level_3_mean_CRPS,0.201 +/- 0.0,0.1536 +/- 0.0082,0.2238 +/- 0.0074,0.2076 +/- 0.0187
level_4_mean_CRPS,0.2399 +/- 0.0,0.1711 +/- 0.0067,0.2555 +/- 0.0109,0.2567 +/- 0.0205
level_5_mean_CRPS,0.3506 +/- 0.0,0.3047 +/- 0.0076,0.3663 +/- 0.0047,0.4047 +/- 0.0223


## Supplementary material tables

In [11]:
for dataset in ['tourism', 'tourismlarge', 'labour', 'traffic', 'wiki']:
    print(f'\n\nThis is a table displaying the CRPS score across levels \non the dataset {dataset} for the optimal configuration of each method: \n')
    display(get_supplementary_table(dataset, to_latex=False))



This is a table displaying the CRPS score across levels 
on the dataset tourism for the optimal configuration of each method: 



,mean_CRPS,level_1_mean_CRPS,level_2_mean_CRPS,level_3_mean_CRPS,level_4_mean_CRPS
ARIMA_NaiveBU,0.1138 +/- 0.0,0.0588 +/- 0.0,0.0945 +/- 0.0,0.1366 +/- 0.0,0.1653 +/- 0.0
ETS_NaiveBU,0.1008 +/- 0.0,0.0545 +/- 0.0,0.0809 +/- 0.0,0.1194 +/- 0.0,0.1483 +/- 0.0
ARIMA_MINT_shr,0.1171 +/- 0.0,0.0625 +/- 0.0,0.0989 +/- 0.0,0.1395 +/- 0.0,0.1677 +/- 0.0
ARIMA_MINT_ols,0.1195 +/- 0.0,0.0619 +/- 0.0,0.1018 +/- 0.0,0.1419 +/- 0.0,0.1723 +/- 0.0
ETS_MINT_shr,0.1013 +/- 0.0,0.0592 +/- 0.0,0.0793 +/- 0.0,0.1202 +/- 0.0,0.1467 +/- 0.0
ETS_MINT_ols,0.1002 +/- 0.0,0.0597 +/- 0.0,0.0749 +/- 0.0,0.1201 +/- 0.0,0.1462 +/- 0.0
ARIMA_ERM,0.5885 +/- 0.0,0.2196 +/- 0.0,0.3903 +/- 0.0,0.812 +/- 0.0,0.9322 +/- 0.0
ETS_ERM,2.3742 +/- 0.0,1.4383 +/- 0.0,1.9934 +/- 0.0,2.8479 +/- 0.0,3.2173 +/- 0.0
PERMBU_MINT,0.0763 +/- 0.0003,0.0464 +/- 0.0017,0.0592 +/- 0.0008,0.0899 +/- 0.0011,0.1097 +/- 0.0009
HierE2E,0.0916 +/- 0.0091,0.051 +/- 0.0099,0.0765 +/- 0.0113,0.1104 +/- 0.008,0.1286 +/- 0.0079




This is a table displaying the CRPS score across levels 
on the dataset tourismlarge for the optimal configuration of each method: 



,mean_CRPS,level_1_mean_CRPS,level_2_geo_mean_CRPS,level_3_geo_mean_CRPS,level_4_geo_mean_CRPS,level_2_trav_mean_CRPS,level_3_trav_mean_CRPS,level_4_trav_mean_CRPS,level_5_trav_mean_CRPS
ARIMA_NaiveBU,0.1752 +/- 0.0,0.0827 +/- 0.0,0.1035 +/- 0.0,0.1586 +/- 0.0,0.2131 +/- 0.0,0.1003 +/- 0.0,0.1567 +/- 0.0,0.2489 +/- 0.0,0.3379 +/- 0.0
ETS_NaiveBU,0.169 +/- 0.0,0.0802 +/- 0.0,0.0989 +/- 0.0,0.1561 +/- 0.0,0.2058 +/- 0.0,0.0927 +/- 0.0,0.1484 +/- 0.0,0.2408 +/- 0.0,0.3291 +/- 0.0
ARIMA_MINT_shr,0.1615 +/- 0.0,0.0443 +/- 0.0,0.0826 +/- 0.0,0.1439 +/- 0.0,0.2042 +/- 0.0,0.0834 +/- 0.0,0.1485 +/- 0.0,0.244 +/- 0.0,0.3413 +/- 0.0
ARIMA_MINT_ols,0.1731 +/- 0.0,0.0394 +/- 0.0,0.083 +/- 0.0,0.1501 +/- 0.0,0.2169 +/- 0.0,0.1056 +/- 0.0,0.1646 +/- 0.0,0.261 +/- 0.0,0.3643 +/- 0.0
ETS_MINT_shr,0.1627 +/- 0.0,0.0505 +/- 0.0,0.0902 +/- 0.0,0.1501 +/- 0.0,0.2024 +/- 0.0,0.089 +/- 0.0,0.1439 +/- 0.0,0.2415 +/- 0.0,0.3343 +/- 0.0
ETS_MINT_ols,0.1668 +/- 0.0,0.0484 +/- 0.0,0.0897 +/- 0.0,0.1542 +/- 0.0,0.2102 +/- 0.0,0.0891 +/- 0.0,0.1455 +/- 0.0,0.2499 +/- 0.0,0.3473 +/- 0.0
ARIMA_ERM,0.5668 +/- 0.0,0.2577 +/- 0.0,0.3791 +/- 0.0,0.4974 +/- 0.0,0.638 +/- 0.0,0.366 +/- 0.0,0.5402 +/- 0.0,0.8013 +/- 0.0,1.0551 +/- 0.0
ETS_ERM,0.508 +/- 0.0,0.1161 +/- 0.0,0.3231 +/- 0.0,0.4684 +/- 0.0,0.6143 +/- 0.0,0.2622 +/- 0.0,0.4853 +/- 0.0,0.7741 +/- 0.0,1.0209 +/- 0.0
PERMBU_MINT,NA,NA,NA,NA,NA,NA,NA,NA,NA
HierE2E,0.1688 +/- 0.004,0.0959 +/- 0.0105,0.1161 +/- 0.0063,0.1503 +/- 0.0053,0.1901 +/- 0.0045,0.1209 +/- 0.0039,0.1619 +/- 0.0044,0.2242 +/- 0.0044,0.2913 +/- 0.0053




This is a table displaying the CRPS score across levels 
on the dataset labour for the optimal configuration of each method: 



,mean_CRPS,level_1_mean_CRPS,level_2_mean_CRPS,level_3_mean_CRPS,level_4_mean_CRPS
ARIMA_NaiveBU,0.0453 +/- 0.0,0.0437 +/- 0.0,0.0441 +/- 0.0,0.0447 +/- 0.0,0.0489 +/- 0.0
ETS_NaiveBU,0.0432 +/- 0.0,0.0416 +/- 0.0,0.0418 +/- 0.0,0.0421 +/- 0.0,0.0471 +/- 0.0
ARIMA_MINT_shr,0.0467 +/- 0.0,0.0454 +/- 0.0,0.0455 +/- 0.0,0.0459 +/- 0.0,0.0499 +/- 0.0
ARIMA_MINT_ols,0.0463 +/- 0.0,0.0448 +/- 0.0,0.045 +/- 0.0,0.0455 +/- 0.0,0.0499 +/- 0.0
ETS_MINT_shr,0.0455 +/- 0.0,0.044 +/- 0.0,0.0442 +/- 0.0,0.0444 +/- 0.0,0.0492 +/- 0.0
ETS_MINT_ols,0.0459 +/- 0.0,0.0445 +/- 0.0,0.0447 +/- 0.0,0.0448 +/- 0.0,0.0495 +/- 0.0
ARIMA_ERM,0.0399 +/- 0.0,0.0365 +/- 0.0,0.0379 +/- 0.0,0.0391 +/- 0.0,0.0459 +/- 0.0
ETS_ERM,0.0456 +/- 0.0,0.0409 +/- 0.0,0.0437 +/- 0.0,0.0452 +/- 0.0,0.0525 +/- 0.0
PERMBU_MINT,0.0393 +/- 0.0003,0.0406 +/- 0.0004,0.0388 +/- 0.0003,0.0382 +/- 0.0002,0.0396 +/- 0.0003
HierE2E,0.0335 +/- 0.0072,0.0302 +/- 0.0093,0.0342 +/- 0.0071,0.0335 +/- 0.0066,0.0361 +/- 0.0058




This is a table displaying the CRPS score across levels 
on the dataset traffic for the optimal configuration of each method: 



,mean_CRPS,level_1_mean_CRPS,level_2_mean_CRPS,level_3_mean_CRPS,level_4_mean_CRPS
ARIMA_NaiveBU,0.0753 +/- 0.0,0.0364 +/- 0.0,0.0364 +/- 0.0,0.0453 +/- 0.0,0.1832 +/- 0.0
ETS_NaiveBU,0.0665 +/- 0.0,0.0128 +/- 0.0,0.0128 +/- 0.0,0.0351 +/- 0.0,0.2053 +/- 0.0
ARIMA_MINT_shr,0.0775 +/- 0.0,0.0467 +/- 0.0,0.0467 +/- 0.0,0.0467 +/- 0.0,0.1701 +/- 0.0
ARIMA_MINT_ols,0.1123 +/- 0.0,0.0853 +/- 0.0,0.0853 +/- 0.0,0.0853 +/- 0.0,0.1934 +/- 0.0
ETS_MINT_shr,0.0963 +/- 0.0,0.0601 +/- 0.0,0.0601 +/- 0.0,0.0601 +/- 0.0,0.205 +/- 0.0
ETS_MINT_ols,0.111 +/- 0.0,0.0765 +/- 0.0,0.0765 +/- 0.0,0.0765 +/- 0.0,0.2145 +/- 0.0
ARIMA_ERM,0.0466 +/- 0.0,0.0089 +/- 0.0,0.0113 +/- 0.0,0.0254 +/- 0.0,0.1408 +/- 0.0
ETS_ERM,0.1027 +/- 0.0,0.0828 +/- 0.0,0.0828 +/- 0.0,0.0828 +/- 0.0,0.1624 +/- 0.0
PERMBU_MINT,0.0679 +/- 0.0053,0.0346 +/- 0.0072,0.0354 +/- 0.0058,0.0419 +/- 0.0044,0.1598 +/- 0.0042
HierE2E,0.0359 +/- 0.0127,0.0166 +/- 0.017,0.0178 +/- 0.0159,0.0186 +/- 0.0154,0.0905 +/- 0.0061




This is a table displaying the CRPS score across levels 
on the dataset wiki for the optimal configuration of each method: 



,mean_CRPS,level_1_mean_CRPS,level_2_mean_CRPS,level_3_mean_CRPS,level_4_mean_CRPS,level_5_mean_CRPS
ARIMA_NaiveBU,0.3776 +/- 0.0,0.1904 +/- 0.0,0.2797 +/- 0.0,0.4118 +/- 0.0,0.4124 +/- 0.0,0.5936 +/- 0.0
ETS_NaiveBU,0.4673 +/- 0.0,0.341 +/- 0.0,0.3863 +/- 0.0,0.4631 +/- 0.0,0.5051 +/- 0.0,0.641 +/- 0.0
ARIMA_MINT_shr,0.2466 +/- 0.0,0.08 +/- 0.0,0.1382 +/- 0.0,0.2559 +/- 0.0,0.2953 +/- 0.0,0.4638 +/- 0.0
ARIMA_MINT_ols,0.2782 +/- 0.0,0.1079 +/- 0.0,0.1743 +/- 0.0,0.2857 +/- 0.0,0.3253 +/- 0.0,0.4977 +/- 0.0
ETS_MINT_shr,0.3622 +/- 0.0,0.218 +/- 0.0,0.2666 +/- 0.0,0.3451 +/- 0.0,0.388 +/- 0.0,0.5936 +/- 0.0
ETS_MINT_ols,0.2702 +/- 0.0,0.0234 +/- 0.0,0.1456 +/- 0.0,0.2616 +/- 0.0,0.3138 +/- 0.0,0.6065 +/- 0.0
ARIMA_ERM,0.2195 +/- 0.0,0.0776 +/- 0.0,0.1213 +/- 0.0,0.2325 +/- 0.0,0.2746 +/- 0.0,0.3913 +/- 0.0
ETS_ERM,0.2217 +/- 0.0,0.1558 +/- 0.0,0.1614 +/- 0.0,0.201 +/- 0.0,0.2399 +/- 0.0,0.3506 +/- 0.0
PERMBU_MINT,0.279 +/- 0.0223,0.094 +/- 0.0394,0.1599 +/- 0.0248,0.2689 +/- 0.0293,0.3056 +/- 0.0305,0.5666 +/- 0.0589
HierE2E,0.1629 +/- 0.0063,0.0668 +/- 0.0056,0.1184 +/- 0.0062,0.1536 +/- 0.0082,0.1711 +/- 0.0067,0.3047 +/- 0.0076


## Calculation of average runtime of optimal configurations

In [18]:
runtime_df = calculate_average_runtimes(to_latex=False, calc_std_error=True, minutes=True)
runtime_df

,labour,traffic,tourism,tourismlarge,wiki
ARIMA_NaiveBU,2.61 +/- 0.23,9.43 +/- 0.14,0.35 +/- 0.01,10.57 +/- 0.63,2.26 +/- 0.16
ETS_NaiveBU,1.73 +/- 0.38,3.42 +/- 0.17,0.32 +/- 0.01,4.11 +/- 0.05,2.11 +/- 0.14
ARIMA_MINT_shr,3.07 +/- 0.35,13.18 +/- 5.18,0.45 +/- 0.01,29.48 +/- 0.21,2.47 +/- 0.02
ARIMA_MINT_ols,2.69 +/- 0.35,9.71 +/- 0.05,0.47 +/- 0.01,29.51 +/- 0.23,2.46 +/- 0.02
ETS_MINT_shr,2.28 +/- 0.11,3.42 +/- 0.01,0.43 +/- 0.02,8.44 +/- 0.19,2.54 +/- 0.07
ETS_MINT_ols,2.55 +/- 0.02,3.38 +/- 0.01,0.42 +/- 0.03,8.21 +/- 0.01,2.46 +/- 0.04
ARIMA_ERM,24.8 +/- 0.06,13.32 +/- 0.44,6.97 +/- 0.69,435.63 +/- 39.84,3.54 +/- 0.1
ETS_ERM,16.25 +/- 0.15,4.27 +/- 0.22,5.91 +/- 0.12,142.75 +/- 17.88,2.98 +/- 0.13
PERMBU_MINT,3.15 +/- 0.09,7.4 +/- 0.4,1.99 +/- 0.12,None,6.8 +/- 0.08
HierE2E,18.95 +/- 0.01,55.05 +/- 0.4,5.88 +/- 0.03,38.35 +/- 0.61,21.81 +/- 0.02


## Calculation of total experiment time

In [19]:
display(calculate_total_time())

,total_time [h]
ARIMA_NaiveBU,2.10
ETS_NaiveBU,0.97
ARIMA_MINT_shr,4.05
ARIMA_MINT_ols,3.74
ETS_MINT_shr,1.43
ETS_MINT_ols,1.42
ARIMA_ERM,40.36
ETS_ERM,14.35
PERMBU_MINT,1.61
HierE2E,112.28
